## Data : Flight
항공 데이터와 상권 정보를 통한 인사이트 돌출
* 항공 데이터 : 항공정보포털시스템 데이터 취합(https://www.airportal.go.kr/knowledge/statsnew/main.jsp)
* 상권 정보 : 소상공인시장진흥공단_상가(상권)정보_202303(https://www.data.go.kr/data/15083033/fileData.do)

In [1]:
#라이브러리 불러오기
import os 
import sys 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from matplotlib import font_manager 
from datetime import datetime, timedelta
from haversine import haversine, Unit

### 01. 공항 데이터 탐색

In [2]:
# 공항 데이터 불러오기
Airport_csv = pd.read_csv("Flight.csv")

In [3]:
Airport_csv.head()

,Scheduled Date,Flight Plan,Flight Happened,Flight Delayed,Flight Canceled,Delay CD,Delay Desc,Cancel CD,Cancel Desc,Departure / Arrival,...,Country,City,Corresponding Airport CD,Corresponding Airport Lat,Corresponding Airport Long,Passengers Charged,Passengers Transferred,Freight (kg),Baggage (kg),Mail (kg)
0,2019-01-01,1,1,0,0,0,NaN,0,NaN,출발,...,이디오피아,아디스아바바,ADD,8.9779,38.7993,171,1,747,5636,0
1,2019-01-04,1,1,0,0,0,NaN,0,NaN,도착,...,이디오피아,아디스아바바,ADD,8.9779,38.7993,60,0,4052,1491,0
2,2019-01-06,1,1,0,0,0,NaN,0,NaN,도착,...,이디오피아,아디스아바바,ADD,8.9779,38.7993,47,1,4136,1617,0
3,2019-01-10,1,1,0,0,0,NaN,0,NaN,출발,...,이디오피아,아디스아바바,ADD,8.9779,38.7993,152,8,1070,5023,0
4,2019-01-13,1,1,0,0,0,NaN,0,NaN,도착,...,이디오피아,아디스아바바,ADD,8.9779,38.7993,128,11,2525,3720,0


In [4]:
# 31개의 칼럼과 750388개의 행으로 구성되어 있다.
Airport_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750388 entries, 0 to 750387
Data columns (total 32 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Scheduled Date              750388 non-null  object 
 1   Flight Plan                 750388 non-null  int64  
 2   Flight Happened             750388 non-null  int64  
 3   Flight Delayed              750388 non-null  int64  
 4   Flight Canceled             750388 non-null  int64  
 5   Delay CD                    750388 non-null  int64  
 6   Delay Desc                  53515 non-null   object 
 7   Cancel CD                   750388 non-null  int64  
 8   Cancel Desc                 5894 non-null    object 
 9   Departure / Arrival         750388 non-null  object 
 10  Domestic / International    750388 non-null  object 
 11  Regular / Irregular         750388 non-null  object 
 12  Passenger / Freight         750388 non-null  int64  
 13  Airport CD    

### 01-1. 데이터 전처리
분석하고자 하는 주제가 공항 주변 상권별 특징을 파악하고자 했기 때문에 그외에 필요한 

데이터와 불필요한 데이터들을 구분하여 데이터를 가공한다.

In [ ]:
# 불필요한 칼럼들 제거
Airport_csv.drop(['Delay CD', 'Delay Desc', 'Cancel CD', 'Cancel Desc', 'Departure / Arrival','Airline Category','Continent Group','Continent Sub-Group', 'Passengers Charged','Corresponding Airport CD',
'Passenger / Freight','Passengers Transferred', 'Freight (kg)', 'Baggage (kg)', 'Mail (kg)'], axis = 1, inplace = True)

In [ ]:
# 가공된 데이터 확인
Airport_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750388 entries, 0 to 750387
Data columns (total 17 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Scheduled Date              750388 non-null  object 
 1   Flight Plan                 750388 non-null  int64  
 2   Flight Happened             750388 non-null  int64  
 3   Flight Delayed              750388 non-null  int64  
 4   Flight Canceled             750388 non-null  int64  
 5   Domestic / International    750388 non-null  object 
 6   Regular / Irregular         750388 non-null  object 
 7   Airport CD                  750388 non-null  object 
 8   Airport NM                  750388 non-null  object 
 9   Airport Lat                 750388 non-null  float64
 10  Airport Long                750388 non-null  float64
 11  Airline ICAO                750384 non-null  object 
 12  Airline NM                  750388 non-null  object 
 13  Country       

In [7]:
Airport_csv.head()

,Scheduled Date,Flight Plan,Flight Happened,Flight Delayed,Flight Canceled,Domestic / International,Regular / Irregular,Airport CD,Airport NM,Airport Lat,Airport Long,Airline ICAO,Airline NM,Country,City,Corresponding Airport Lat,Corresponding Airport Long
0,2019-01-01,1,1,0,0,국제선,정기,ICN,인천,37.4691,126.451,ETH,에티오피아 항공,이디오피아,아디스아바바,8.9779,38.7993
1,2019-01-04,1,1,0,0,국제선,정기,ICN,인천,37.4691,126.451,ETH,에티오피아 항공,이디오피아,아디스아바바,8.9779,38.7993
2,2019-01-06,1,1,0,0,국제선,정기,ICN,인천,37.4691,126.451,ETH,에티오피아 항공,이디오피아,아디스아바바,8.9779,38.7993
3,2019-01-10,1,1,0,0,국제선,정기,ICN,인천,37.4691,126.451,ETH,에티오피아 항공,이디오피아,아디스아바바,8.9779,38.7993
4,2019-01-13,1,1,0,0,국제선,정기,ICN,인천,37.4691,126.451,ETH,에티오피아 항공,이디오피아,아디스아바바,8.9779,38.7993


In [ ]:
Airport_csv['Scheduled Date']

0         2019-01-01
1         2019-01-04
2         2019-01-06
3         2019-01-10
4         2019-01-13
             ...    
750383    2021-11-14
750384    2021-11-21
750385    2021-12-06
750386    2021-12-12
750387    2021-12-13
Name: Scheduled Date, Length: 750388, dtype: object

In [9]:
# 국내 공항만 파악하기 위해 '한국' 데이터만 추출
Airport_csv = Airport_csv.loc[Airport_csv['Country'] == '한국']
Airport_csv.head()

,Scheduled Date,Flight Plan,Flight Happened,Flight Delayed,Flight Canceled,Domestic / International,Regular / Irregular,Airport CD,Airport NM,Airport Lat,Airport Long,Airline ICAO,Airline NM,Country,City,Corresponding Airport Lat,Corresponding Airport Long
39062,2019-01-01,1,1,0,0,국내선,부정기,CJU,제주,33.5113,126.493,JJA,제주항공,한국,청주,36.7166,127.499
39063,2019-01-01,1,1,0,0,국내선,정기,CJU,제주,33.5113,126.493,JJA,제주항공,한국,청주,36.7166,127.499
39064,2019-01-01,1,1,0,0,국내선,정기,CJU,제주,33.5113,126.493,ESR,이스타항공,한국,청주,36.7166,127.499
39065,2019-01-01,1,1,0,0,국내선,정기,CJU,제주,33.5113,126.493,KAL,대한항공,한국,청주,36.7166,127.499
39066,2019-01-01,1,1,0,0,국내선,정기,CJU,제주,33.5113,126.493,JNA,진에어,한국,청주,36.7166,127.499


In [10]:
# 결측치 확인 결과 0개 확인
Airport_csv.isnull().sum()

Scheduled Date                0
Flight Plan                   0
Flight Happened               0
Flight Delayed                0
Flight Canceled               0
Domestic / International      0
Regular / Irregular           0
Airport CD                    0
Airport NM                    0
Airport Lat                   0
Airport Long                  0
Airline ICAO                  0
Airline NM                    0
Country                       0
City                          0
Corresponding Airport Lat     0
Corresponding Airport Long    0
dtype: int64

In [ ]:
# 한국 데이터만 존재하는지 확인
Airport_csv.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 445811 entries, 39062 to 746857
Data columns (total 17 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Scheduled Date              445811 non-null  object 
 1   Flight Plan                 445811 non-null  int64  
 2   Flight Happened             445811 non-null  int64  
 3   Flight Delayed              445811 non-null  int64  
 4   Flight Canceled             445811 non-null  int64  
 5   Domestic / International    445811 non-null  object 
 6   Regular / Irregular         445811 non-null  object 
 7   Airport CD                  445811 non-null  object 
 8   Airport NM                  445811 non-null  object 
 9   Airport Lat                 445811 non-null  float64
 10  Airport Long                445811 non-null  float64
 11  Airline ICAO                445811 non-null  object 
 12  Airline NM                  445811 non-null  object 
 13  Country   

### 02. 상권 분석 데이터 확인
수도권 데이터로 데이터 탐색

In [ ]:
# 경기 상권 데이터 확인
City_gyeong_gi = pd.read_csv("소상공인시장진흥공단_상가(상권)정보_경기_202303.csv")

C:\Users\admin\AppData\Local\Temp\ipykernel_5296\2744861784.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  estate_gyeong_gi = pd.read_csv("소상공인시장진흥공단_상가(상권)정보_경기_202303.csv")


In [19]:
City_gyeong_gi.head()

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
0,MA0101202210A0004063,팽성농산물센터,NaN,G2,소매,G205,식료품 소매,G20506,채소/과일 소매업,G47215,...,4122025039100930001033303,NaN,경기도 평택시 팽성읍 석근2길 36,451803,17997,NaN,NaN,NaN,127.048075,36.936336
1,MA0101202210A0089792,역전할머니맥주평택궁리점,NaN,I2,음식,I211,주점,I21103,생맥주 전문,I56213,...,4122033026100720011024544,NaN,경기도 평택시 고덕면 궁2길 67,451841,17821,NaN,NaN,NaN,127.034381,37.018039
2,MA0101202210A0101847,지제역부동산마트공인중개사사무소,NaN,L1,부동산,L102,부동산 서비스,L10203,부동산 중개/대리업,L68221,...,4122011900100000000000107,NaN,경기도 평택시 지제동삭2로 181-24,450802,18029,NaN,1,NaN,127.093832,37.016832
3,MA0101202210A0050965,혜윰건축사사무소,NaN,M1,과학·기술,M109,기술 서비스,M10901,건축 설계 및 관련 서비스업,M72111,...,4127110100106240012004645,고요한정원,경기도 안산시 상록구 웃말5길 9-1,426860,15328,NaN,NaN,NaN,126.871046,37.306182
4,MA0101202210A0067941,양촌리민물장어,NaN,I2,음식,I201,한식,I20112,해산물 구이/찜,I56111,...,4148010800109200000000001,NaN,경기도 파주시 경의로 968,413200,10906,NaN,NaN,NaN,126.759748,37.703802


In [13]:
# 서울 데이터 확인
City_seoul = pd.read_csv("소상공인시장진흥공단_상가(상권)정보_서울_202303.csv")

C:\Users\admin\AppData\Local\Temp\ipykernel_5296\3528591632.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  estate_read = pd.read_csv("소상공인시장진흥공단_상가(상권)정보_서울_202303.csv")


In [14]:
City_seoul.head()

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
0,MA0101202210A0037544,영자's곱창,NaN,I2,음식,I201,한식,I20109,곱창 전골/구이,I56111,...,1121510100101690021003140,NaN,서울특별시 광진구 면목로 172,143900,4904,NaN,1,NaN,127.082832,37.568503
1,MA0101202210A0097916,호텔클로이,NaN,I1,숙박,I101,일반 숙박,I10102,여관/모텔,I55102,...,1114014500101300000014092,유성장,서울특별시 중구 을지로42길 11,100411,4561,NaN,NaN,NaN,127.005849,37.565708
2,MA0101202210A0096343,나라차중계태권도장,NaN,P1,교육,P106,기타 교육,P10601,태권도/무술학원,P85611,...,1135010600103640019010971,삼부프라자,서울특별시 노원구 한글비석로 242,139861,1734,NaN,NaN,NaN,127.076956,37.649226
3,MA0101202210A0093845,이상한스냅,NaN,M1,과학·기술,M113,사진 촬영,M11301,사진촬영업,M73303,...,1147010300100090004000001,백송주택,서울특별시 양천구 월정로 283,158822,7902,NaN,NaN,NaN,126.828832,37.542117
4,MA010120220805430767,토끼정,NaN,I2,음식,I201,한식,I20101,백반/한정식,I56111,...,1150010500107740012000001,마커스빌딩,서울특별시 강서구 마곡동로 55,157805,7802,NaN,2,NaN,126.833022,37.560053


In [21]:
# 수도권 데이터 비교
City_gyeong_gi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 554709 entries, 0 to 554708
Data columns (total 39 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   상가업소번호     554709 non-null  object 
 1   상호명        554709 non-null  object 
 2   지점명        1794 non-null    object 
 3   상권업종대분류코드  554709 non-null  object 
 4   상권업종대분류명   554709 non-null  object 
 5   상권업종중분류코드  554709 non-null  object 
 6   상권업종중분류명   554709 non-null  object 
 7   상권업종소분류코드  554709 non-null  object 
 8   상권업종소분류명   554709 non-null  object 
 9   표준산업분류코드   554709 non-null  object 
 10  표준산업분류명    554709 non-null  object 
 11  시도코드       554709 non-null  int64  
 12  시도명        554709 non-null  object 
 13  시군구코드      554709 non-null  int64  
 14  시군구명       554709 non-null  object 
 15  행정동코드      554709 non-null  int64  
 16  행정동명       554709 non-null  object 
 17  법정동코드      554709 non-null  int64  
 18  법정동명       554709 non-null  object 
 19  지번코드       554709 non-n

In [15]:
City_seoul.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 474260 entries, 0 to 474259
Data columns (total 39 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   상가업소번호     474260 non-null  object 
 1   상호명        474260 non-null  object 
 2   지점명        1216 non-null    object 
 3   상권업종대분류코드  474260 non-null  object 
 4   상권업종대분류명   474260 non-null  object 
 5   상권업종중분류코드  474260 non-null  object 
 6   상권업종중분류명   474260 non-null  object 
 7   상권업종소분류코드  474260 non-null  object 
 8   상권업종소분류명   474260 non-null  object 
 9   표준산업분류코드   474260 non-null  object 
 10  표준산업분류명    474260 non-null  object 
 11  시도코드       474260 non-null  int64  
 12  시도명        474260 non-null  object 
 13  시군구코드      474260 non-null  int64  
 14  시군구명       474260 non-null  object 
 15  행정동코드      474260 non-null  int64  
 16  행정동명       474260 non-null  object 
 17  법정동코드      474260 non-null  int64  
 18  법정동명       474260 non-null  object 
 19  지번코드       474260 non-n

### 02-1. 상권 분석 데이터 전처리


In [ ]:
path = "./"
file_list = os.listdir(path)
file_list_csv = [file for file in file_list if file.endswith("3.csv")]
file_list_csv

In [ ]:
# 공항별 연고지를 리스트에 저장
Airport_City_list = ['인천광역시 중구','서울특별시 강서구','횡성읍','원주시 소초면','양양군','군산시','청주시','대구광역시 동구','포항시','울산광역시 북구','부산광역시 강서구',
                     '부산광역시 사하구','부산광역시 사상구','사천시','여수시','제주시','무안군','광주광역시 광산구']
# 상권분석 2023.03 일자 파일만 골라 리스트에 저장
path = "./"
file_list = os.listdir(path)
file_list_csv = [file for file in file_list if file.endswith("3.csv")]
file_list_csv

['소상공인시장진흥공단_상가(상권)정보_강원_202303.csv',
 '소상공인시장진흥공단_상가(상권)정보_경기_202303.csv',
 '소상공인시장진흥공단_상가(상권)정보_경남_202303.csv',
 '소상공인시장진흥공단_상가(상권)정보_경북_202303.csv',
 '소상공인시장진흥공단_상가(상권)정보_광주_202303.csv',
 '소상공인시장진흥공단_상가(상권)정보_대구_202303.csv',
 '소상공인시장진흥공단_상가(상권)정보_대전_202303.csv',
 '소상공인시장진흥공단_상가(상권)정보_부산_202303.csv',
 '소상공인시장진흥공단_상가(상권)정보_서울_202303.csv',
 '소상공인시장진흥공단_상가(상권)정보_세종_202303.csv',
 '소상공인시장진흥공단_상가(상권)정보_울산_202303.csv',
 '소상공인시장진흥공단_상가(상권)정보_인천_202303.csv',
 '소상공인시장진흥공단_상가(상권)정보_전남_202303.csv',
 '소상공인시장진흥공단_상가(상권)정보_전북_202303.csv',
 '소상공인시장진흥공단_상가(상권)정보_제주_202303.csv',
 '소상공인시장진흥공단_상가(상권)정보_충남_202303.csv',
 '소상공인시장진흥공단_상가(상권)정보_충북_202303.csv']

In [ ]:

# 상권 분석 리스트와 연고지 리스트를 통해 원하는 주소지 데이터들만 필터링
def Data_All_EDA(csv_path, Airport_City_list):
    data1 = pd.DataFrame()
    for i in range(len(csv_path)):
        data = pd.read_csv(csv_path[i])
        data.drop(columns= ['상가업소번호', '지점명', '상권업종중분류코드', '표준산업분류코드', '시도코드', '시군구코드', '행정동코드', '행정동명', '법정동코드', '법정동명', '지번코드'
        , '대지구분코드', '대지구분명', '지번본번지', '도로명코드', '지번주소', '건물본번지', '도로명', '건물본번지', '건물부번지', '건물관리번호', '건물명', '도로명주소'
        '구우편번호', '신우편번호', '동정보', '층정보', '호정보','지번부번지','구우편번호','지번부번지','상권업종소분류코드','상권업종대분류코드'], axis=1 , inplace=True, errors='ignore')
        for list in Airport_City_list:
            data2 = data[data['도로명주소'].str.contains(list)]
            data2['Airport_City'] = list
            data1 = pd.concat([data1,data2])
        data2
    return data1

result_Data = Data_All_EDA(file_list_csv, Airport_City_list)

C:\Users\admin\AppData\Local\Temp\ipykernel_9220\4114227930.py:12: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(csv_path[i])
C:\Users\admin\AppData\Local\Temp\ipykernel_9220\4114227930.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Airport_City'] = list
C:\Users\admin\AppData\Local\Temp\ipykernel_9220\4114227930.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Airport_City'] = list
C:\Users\admin\AppData\Loca

In [ ]:
# 합쳐진 데이터 확인
result_Data.head()

,상호명,상권업종대분류명,상권업종중분류명,상권업종소분류명,표준산업분류명,시도명,시군구명,도로명주소,경도,위도,Airport_City
245,궁,음식,주점,일반 유흥 주점,일반 유흥 주점업,강원도,횡성군,강원도 횡성군 횡성읍 횡성로 407,127.985873,37.489219,횡성읍
636,현대광고,과학·기술,인쇄·제품제작,명함/간판/광고물 제작,간판 및 광고물 제조업,강원도,횡성군,강원도 횡성군 횡성읍 문정로 20,127.985545,37.489795,횡성읍
653,이숙자카페,음식,주점,생맥주 전문,생맥주 전문점,강원도,횡성군,강원도 횡성군 횡성읍 삼일로 4,127.986596,37.489814,횡성읍
754,토탈명품,소매,섬유·의복·신발 소매,기타 의류 소매업,셔츠 및 블라우스 소매업,강원도,횡성군,강원도 횡성군 횡성읍 문정로 23-1,127.986235,37.489796,횡성읍
808,제이서플러스,소매,장식품 소매,기념품점,"기념품, 관광 민예품 및 장식용품 소매업",강원도,횡성군,강원도 횡성군 횡성읍 태기로 488,128.026965,37.512924,횡성읍


In [ ]:
result_Data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 250780 entries, 245 to 83554
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   상호명           250780 non-null  object 
 1   상권업종대분류명      250780 non-null  object 
 2   상권업종중분류명      250780 non-null  object 
 3   상권업종소분류명      250780 non-null  object 
 4   표준산업분류명       250780 non-null  object 
 5   시도명           250780 non-null  object 
 6   시군구명          250780 non-null  object 
 7   도로명주소         250780 non-null  object 
 8   경도            250780 non-null  float64
 9   위도            250780 non-null  float64
 10  Airport_City  250780 non-null  object 
dtypes: float64(2), object(9)
memory usage: 23.0+ MB


In [ ]:
# 결측치 확인
result_Data.isnull().sum()

상호명             0
상권업종대분류명        0
상권업종중분류명        0
상권업종소분류명        0
표준산업분류명         0
시도명             0
시군구명            0
도로명주소           0
경도              0
위도              0
Airport_City    0
dtype: int64

In [ ]:
# 시별 데이터 확인
result_Data['시도명'].unique()

array(['강원도', '경상남도', '경상북도', '광주광역시', '대구광역시', '부산광역시', '서울특별시', '울산광역시',
       '인천광역시', '전라남도', '전라북도', '제주특별자치도', '충청북도'], dtype=object)

In [ ]:
# 상권 분석 데이터의 '시도명' 데이터 확인
result_Data.loc[result_Data['시도명'] == '광주광역시']

,상호명,상권업종대분류명,상권업종중분류명,상권업종소분류명,표준산업분류명,시도명,시군구명,도로명주소,경도,위도,Airport_City
1,몰카페,음식,비알코올,카페,커피 전문점,광주광역시,광산구,광주광역시 광산구 사암로 790,126.818663,35.213047,광주광역시 광산구
2,명품고로케NO.1,음식,기타 간이,빵/도넛,제과점업,광주광역시,광산구,광주광역시 광산구 소촌로86번길 11,126.794245,35.152315,광주광역시 광산구
14,호초밥,음식,일식,일식 회/초밥,일식 음식점업,광주광역시,광산구,광주광역시 광산구 수등로94번길 26,126.819033,35.182233,광주광역시 광산구
22,스팀에듀STEAMEDU어학원,교육,일반 교육,입시·교과학원,일반 교과학원,광주광역시,광산구,광주광역시 광산구 목련로253번길 32,126.823745,35.174454,광주광역시 광산구
23,빛고을청아보리사랑,소매,종합 소매,슈퍼마켓,"빵류, 과자류 및 당류 소매업",광주광역시,광산구,광주광역시 광산구 평동매화길 52,126.761510,35.141204,광주광역시 광산구
...,...,...,...,...,...,...,...,...,...,...,...
72823,우상인,소매,식료품 소매,정육점,육류 소매업,광주광역시,광산구,광주광역시 광산구 남동길 3,126.789689,35.127932,광주광역시 광산구
72826,로얄,음식,주점,생맥주 전문,생맥주 전문점,광주광역시,광산구,광주광역시 광산구 풍영로 229,126.814007,35.191900,광주광역시 광산구
72833,마법네일&뷰티,수리·개인,이용·미용,네일숍,기타 미용업,광주광역시,광산구,광주광역시 광산구 산월로 25,126.840083,35.208982,광주광역시 광산구
72836,나이스계절포차,음식,주점,일반 유흥 주점,한식 일반 음식점업,광주광역시,광산구,광주광역시 광산구 수완로11번길 33,126.826684,35.186147,광주광역시 광산구


In [ ]:
# 공항 데이터의 'Airport_City' 확인
result_Data['Airport_City'].unique()

array(['횡성읍', '원주시 소초면', '양양군', '사천시', '포항시', '광주광역시 광산구', '대구광역시 동구',
       '부산광역시 강서구', '부산광역시 사하구', '부산광역시 사상구', '서울특별시 강서구', '울산광역시 북구',
       '인천광역시 중구', '여수시', '무안군', '군산시', '제주시', '청주시'], dtype=object)

In [ ]:
# Airport 칼럼을 새로 만들어 공항 데이터 추가
result_Data ['Airport'] = result_Data['Airport_City']
result_Data.head()

,상호명,상권업종대분류명,상권업종중분류명,상권업종소분류명,표준산업분류명,시도명,시군구명,도로명주소,경도,위도,Airport_City,Airport
245,궁,음식,주점,일반 유흥 주점,일반 유흥 주점업,강원도,횡성군,강원도 횡성군 횡성읍 횡성로 407,127.985873,37.489219,횡성읍,횡성읍
636,현대광고,과학·기술,인쇄·제품제작,명함/간판/광고물 제작,간판 및 광고물 제조업,강원도,횡성군,강원도 횡성군 횡성읍 문정로 20,127.985545,37.489795,횡성읍,횡성읍
653,이숙자카페,음식,주점,생맥주 전문,생맥주 전문점,강원도,횡성군,강원도 횡성군 횡성읍 삼일로 4,127.986596,37.489814,횡성읍,횡성읍
754,토탈명품,소매,섬유·의복·신발 소매,기타 의류 소매업,셔츠 및 블라우스 소매업,강원도,횡성군,강원도 횡성군 횡성읍 문정로 23-1,127.986235,37.489796,횡성읍,횡성읍
808,제이서플러스,소매,장식품 소매,기념품점,"기념품, 관광 민예품 및 장식용품 소매업",강원도,횡성군,강원도 횡성군 횡성읍 태기로 488,128.026965,37.512924,횡성읍,횡성읍


In [ ]:
# 변경할 데이터들을 딕셔너리를 통해 해당되는 공항명으로 변경
Airport_Sort = {'인천광역시 중구' : '인천공항', '서울특별시 강서구' : '김포공항' , '횡성읍' : '원주공항' ,'원주시 소초면' : '원주공항' ,'양양군' : '양양공항'
                ,'청주시' : '청주공항', '군산시' : '군산공항', '광주광역시 광산구' : '광주공항', '무안군' : '무안공항' 
                ,'여수시' : '여수공항', '사천시' : '사천공항', '부산광역시 강서구' : '김해공항','부산광역시 사하구' : '김해공항','부산광역시 사상구' : '김해공항' 
                ,'대구광역시 동구' : '대구공항', '포항시' : '포항공항', '울산광역시 북구' : '울산공항','제주시' : '제주공항'}


result_Data['Airport']= result_Data['Airport'].replace(Airport_Sort)

In [ ]:
# Airport 변경 후 데이터 확인
result_Data.head()

,상호명,상권업종대분류명,상권업종중분류명,상권업종소분류명,표준산업분류명,시도명,시군구명,도로명주소,경도,위도,Airport_City,Airport
245,궁,음식,주점,일반 유흥 주점,일반 유흥 주점업,강원도,횡성군,강원도 횡성군 횡성읍 횡성로 407,127.985873,37.489219,횡성읍,원주공항
636,현대광고,과학·기술,인쇄·제품제작,명함/간판/광고물 제작,간판 및 광고물 제조업,강원도,횡성군,강원도 횡성군 횡성읍 문정로 20,127.985545,37.489795,횡성읍,원주공항
653,이숙자카페,음식,주점,생맥주 전문,생맥주 전문점,강원도,횡성군,강원도 횡성군 횡성읍 삼일로 4,127.986596,37.489814,횡성읍,원주공항
754,토탈명품,소매,섬유·의복·신발 소매,기타 의류 소매업,셔츠 및 블라우스 소매업,강원도,횡성군,강원도 횡성군 횡성읍 문정로 23-1,127.986235,37.489796,횡성읍,원주공항
808,제이서플러스,소매,장식품 소매,기념품점,"기념품, 관광 민예품 및 장식용품 소매업",강원도,횡성군,강원도 횡성군 횡성읍 태기로 488,128.026965,37.512924,횡성읍,원주공항


In [ ]:
result_Data['Airport'].isnull().sum()

0

In [ ]:
# 공항이 전부 잘 들어 갔는지 확인
result_Data['Airport'].unique()

array(['원주공항', '양양공항', '사천공항', '포항공항', '광주공항', '대구공항', '김해공항', '김포공항',
       '울산공항', '인천공항', '여수공항', '무안공항', '군산공항', '제주공항', '청주공항'],
      dtype=object)

In [ ]:
# 결과 데이터 칼럼 갯수 확인
result_Data['상권업종중분류명'].nunique()

74

In [ ]:
result_Data['표준산업분류명'].nunique()

334

In [ ]:
result_Data['상권업종대분류명'].nunique()

10

In [ ]:
result_Data['Airport'].value_counts()

청주공항    41455
제주공항    38465
김해공항    26918
포항공항    26588
김포공항    23419
광주공항    17928
여수공항    16023
대구공항    14812
군산공항    14175
인천공항     8822
울산공항     7523
사천공항     6096
무안공항     4216
양양공항     2580
원주공항     1760
Name: Airport, dtype: int64

## 03. 공항 위도 경도
추가적으로 궁금했던 업종별로 공항과의 거리에 대한 관계성이 있는지 파악해 보았다.

In [ ]:
Airport_csv = pd.read_csv("Flight.csv")

In [ ]:
# 공항에서의 상권별 거리를 파악하기 위해 공항 이름과 위도,경도 값을 뺀 나머지 칼러 제거
Airport_csv.drop(['Delay CD', 'Delay Desc', 'Cancel CD', 'Cancel Desc', 'Departure / Arrival','Airline Category','Continent Group','Continent Sub-Group', 'Passengers Charged','Corresponding Airport CD',
'Passenger / Freight','Passengers Transferred', 'Freight (kg)', 'Baggage (kg)', 'Mail (kg)', 'Scheduled Date', 'Flight Plan', 'Flight Happened', 'Flight Delayed', 'Flight Canceled', 'Domestic / International', 'Regular / Irregular',
'Airport CD', 'Airline ICAO', 'Airline NM', 'Country','City', 'Corresponding Airport Lat', 'Corresponding Airport Long'], axis = 1, inplace = True, errors='ignore')

In [ ]:
Airport_csv.head()

,Airport NM,Airport Lat,Airport Long
0,인천,37.4691,126.451
1,인천,37.4691,126.451
2,인천,37.4691,126.451
3,인천,37.4691,126.451
4,인천,37.4691,126.451


In [ ]:
# 공항이 있는 연고지 확인
Airport_csv['Airport NM'].unique()

array(['인천', '청주', '무안', '김해', '대구', '제주', '김포', '사천', '포항', '군산', '광주',
       '여수', '울산', '원주', '양양'], dtype=object)

In [ ]:
# 위도 경도 데이터를 만들어 놓은 csv 파일에 조인하기 위해 공항으로 분류
Airport_csv['Airport NM'] = Airport_csv['Airport NM'] + '공항'
Airport_csv['Airport NM'].unique()

array(['인천공항', '청주공항', '무안공항', '김해공항', '대구공항', '제주공항', '김포공항', '사천공항',
       '포항공항', '군산공항', '광주공항', '여수공항', '울산공항', '원주공항', '양양공항'],
      dtype=object)

In [ ]:
# 조인을 위한 칼럼명 변경
Airport_csv = Airport_csv.rename(columns={'Airport NM':'Airport'})
Join_Airport_csv = Airport_csv.groupby(Airport_csv['Airport']).mean().reset_index()
Join_Airport_csv.head()

,Airport,Airport Lat,Airport Long
0,광주공항,35.1264,126.809
1,군산공항,35.9038,126.616
2,김포공항,37.5583,126.791
3,김해공항,35.1795,128.938
4,대구공항,35.8941,128.659


In [ ]:
result_Data.head()

,상호명,상권업종대분류명,상권업종중분류명,상권업종소분류명,표준산업분류명,시도명,시군구명,도로명주소,경도,위도,Airport_City,Airport
245,궁,음식,주점,일반 유흥 주점,일반 유흥 주점업,강원도,횡성군,강원도 횡성군 횡성읍 횡성로 407,127.985873,37.489219,횡성읍,원주공항
636,현대광고,과학·기술,인쇄·제품제작,명함/간판/광고물 제작,간판 및 광고물 제조업,강원도,횡성군,강원도 횡성군 횡성읍 문정로 20,127.985545,37.489795,횡성읍,원주공항
653,이숙자카페,음식,주점,생맥주 전문,생맥주 전문점,강원도,횡성군,강원도 횡성군 횡성읍 삼일로 4,127.986596,37.489814,횡성읍,원주공항
754,토탈명품,소매,섬유·의복·신발 소매,기타 의류 소매업,셔츠 및 블라우스 소매업,강원도,횡성군,강원도 횡성군 횡성읍 문정로 23-1,127.986235,37.489796,횡성읍,원주공항
808,제이서플러스,소매,장식품 소매,기념품점,"기념품, 관광 민예품 및 장식용품 소매업",강원도,횡성군,강원도 횡성군 횡성읍 태기로 488,128.026965,37.512924,횡성읍,원주공항


In [ ]:
Join_Airport_csv['Airport'].unique()

array(['광주공항', '군산공항', '김포공항', '김해공항', '대구공항', '무안공항', '사천공항', '양양공항',
       '여수공항', '울산공항', '원주공항', '인천공항', '제주공항', '청주공항', '포항공항'],
      dtype=object)

In [ ]:
# 'Airport'칼럼을 기준으로 merge 함수를 사용하기 위해 저장 
merge_keys = Join_Airport_csv['Airport'].unique()
merge_keys

array(['광주공항', '군산공항', '김포공항', '김해공항', '대구공항', '무안공항', '사천공항', '양양공항',
       '여수공항', '울산공항', '원주공항', '인천공항', '제주공항', '청주공항', '포항공항'],
      dtype=object)

In [ ]:
merge_keys = merge_keys.tolist()
merge_keys

['원주공항',
 '양양공항',
 '사천공항',
 '포항공항',
 '광주공항',
 '대구공항',
 '김해공항',
 '김포공항',
 '울산공항',
 '인천공항',
 '여수공항',
 '무안공항',
 '군산공항',
 '제주공항',
 '청주공항']

In [ ]:
# result_Data, Join_Airport_csv 두 개의 데이터를 합쳐 위도, 경도 데이터를 추가해준다.
ALL_Data = pd.merge(result_Data, Join_Airport_csv, on = 'Airport', how='left')
ALL_Data.head()

,상호명,상권업종대분류명,상권업종중분류명,상권업종소분류명,표준산업분류명,시도명,시군구명,도로명주소,경도,위도,Airport_City,Airport,Airport Lat,Airport Long
0,궁,음식,주점,일반 유흥 주점,일반 유흥 주점업,강원도,횡성군,강원도 횡성군 횡성읍 횡성로 407,127.985873,37.489219,횡성읍,원주공항,37.4381,127.96
1,현대광고,과학·기술,인쇄·제품제작,명함/간판/광고물 제작,간판 및 광고물 제조업,강원도,횡성군,강원도 횡성군 횡성읍 문정로 20,127.985545,37.489795,횡성읍,원주공항,37.4381,127.96
2,이숙자카페,음식,주점,생맥주 전문,생맥주 전문점,강원도,횡성군,강원도 횡성군 횡성읍 삼일로 4,127.986596,37.489814,횡성읍,원주공항,37.4381,127.96
3,토탈명품,소매,섬유·의복·신발 소매,기타 의류 소매업,셔츠 및 블라우스 소매업,강원도,횡성군,강원도 횡성군 횡성읍 문정로 23-1,127.986235,37.489796,횡성읍,원주공항,37.4381,127.96
4,제이서플러스,소매,장식품 소매,기념품점,"기념품, 관광 민예품 및 장식용품 소매업",강원도,횡성군,강원도 횡성군 횡성읍 태기로 488,128.026965,37.512924,횡성읍,원주공항,37.4381,127.96


In [ ]:
# 위도, 경도 데이터 확인
ALL_Data['Airport Lat'].unique()

array([37.4381, 38.0613, 35.0885, 35.9879, 35.1264, 35.8941, 35.1795,
       37.5583, 35.5935, 37.4691, 34.8423, 34.9914, 35.9038, 33.5113,
       36.7166])

In [ ]:
# haversine 함수를 통해 위도, 경도 간의 거리를 계산해여 Distance 칼럼에 저장한다.
ALL_Data['Distance'] = ALL_Data.apply(lambda row: haversine((row['위도'], row['경도']),
                                                            (row['Airport Lat'], row['Airport Long']),
                                                            unit=Unit.KILOMETERS), axis=1)

In [ ]:
# Distance 추가 후 데이터 확인
ALL_Data.head()

,상호명,상권업종대분류명,상권업종중분류명,상권업종소분류명,표준산업분류명,시도명,시군구명,도로명주소,경도,위도,Airport_City,Airport,Airport Lat,Airport Long,Distance
0,궁,음식,주점,일반 유흥 주점,일반 유흥 주점업,강원도,횡성군,강원도 횡성군 횡성읍 횡성로 407,127.985873,37.489219,횡성읍,원주공항,37.4381,127.96,6.125696
1,현대광고,과학·기술,인쇄·제품제작,명함/간판/광고물 제작,간판 및 광고물 제조업,강원도,횡성군,강원도 횡성군 횡성읍 문정로 20,127.985545,37.489795,횡성읍,원주공항,37.4381,127.96,6.174608
2,이숙자카페,음식,주점,생맥주 전문,생맥주 전문점,강원도,횡성군,강원도 횡성군 횡성읍 삼일로 4,127.986596,37.489814,횡성읍,원주공항,37.4381,127.96,6.210970
3,토탈명품,소매,섬유·의복·신발 소매,기타 의류 소매업,셔츠 및 블라우스 소매업,강원도,횡성군,강원도 횡성군 횡성읍 문정로 23-1,127.986235,37.489796,횡성읍,원주공항,37.4381,127.96,6.197106
4,제이서플러스,소매,장식품 소매,기념품점,"기념품, 관광 민예품 및 장식용품 소매업",강원도,횡성군,강원도 횡성군 횡성읍 태기로 488,128.026965,37.512924,횡성읍,원주공항,37.4381,127.96,10.205109


In [ ]:
# 전처리 완료 후 파일 추출
ALL_Data.to_csv('Final_Airport.csv')

## 04.Tableau 데이터 시각화
### 1. Tableau 대시보드
<left><img src=".\tableau_picture\T_1.png" width="75%" height="75%"></left>  
총 5 가지의 차트로 구성되어 있으며, 업종별 총 개수 / 카테고리별 업종 개수(전체 개수와 지역별 총 개수) / 지도 차트 / 업종별 평균거리 / 요식업별 평균거리 
구분되어 있다.  
### 2.  Tableau의 대시보드 필터를 적용한 공항별 그래프
<left><img src=".\tableau_picture\T_2.png" width="50%" height="50%"></left><right><img src=".\tableau_picture\T_3.png" width="50%" height="50%"></right>
  ### 3. Tableau의 대시보드 필터를 이용한시 군구별 그래프 
  <left><img src=".\tableau_picture\T_4.png" width="50%" height="50%"></left><right><img src=".\tableau_picture\T_5.png" width="50%" height="50%"></right>
### 4. 데이터 시각화를 통해 얻은 인사이트
* 보통 음식점의 개수가 많았으며, 생각보다 숙박업의 개수가 적었다. 

* 경도, 위도 데이터를 활용하고자 업종별 거리를 파악하였고,  
 숙박 업소는 생각보다 공항에서 멀리 떨어진 경우가 많았다는 것을 알 수 있었다.

* 구내식당을 뺀다면 평균적으로 일식, 한식이 공항에서 가까운 위치를 가지고 있다는 것을 알 수 있다.


<img src=".\python_Project\Mimi\HRhd2Y0.png"></img>